In [ ]:
# # colab설정

# from google.colab import drive
# import os 
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sen
# import numpy as np
# %matplotlib inline

# # display 확장
# pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_rows', 1000)

# # import libraries
# import warnings
# warnings.simplefilter('ignore')

# drive.mount('/content/drive')
# path = '/content/drive/MyDrive/black_swan_physionet/'
# os.chdir(path)

# file = 'train_done_preprocessing.csv'
# file2 = 'valid_done_preprocessing.csv'
# file3 = 'test_done_preprocessing.csv'

# train_df = pd.read_csv(path+file)
# valid_df = pd.read_csv(path+file2)
# test_df = pd.read_csv(path+file3)

# maxlen = len(train_df[train_df['pid'] == train_df['pid'][0]])

# print(len(test_df))
# test_df.head()

In [18]:
# Jupyter Notebook 설정

import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sen
%matplotlib inline
import numpy as np
from google.colab import drive
# import libraries
import warnings
warnings.simplefilter('ignore')

# display 확장
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# path = '/Users/jsha/gjai/final_project/black_swan_physionet/'
drive.mount('/content/drive')
path = '/content/drive/MyDrive/black_swan_physionet/'
os.chdir(path)
file = './test_done_preprocessing.csv'
# file2 = 'physionet_test_data_all_model_pred.csv'

base_test_df = pd.read_csv(file)
# test_all_df = pd.read_csv(path+file2)

maxlen = len(base_test_df[base_test_df['pid'] == base_test_df['pid'][0]])

print(maxlen)
print(len(base_test_df))
print(base_test_df.shape)
# print(test_all_df.shape)
base_test_df.head()

Mounted at /content/drive
70
28000
(28000, 47)


,pid,age,Gender,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,TroponinI,TroponinT,Urine,WBC,pH,MechVent,survival,SBP,DBP,MBP,male,female,surv_los,LOS,labels,time_diff,ICUType_1.0,ICUType_2.0,ICUType_3.0,ICUType_4.0
0,135089.0,90.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,-7.0,6.0,0.0,0.0,0,0,1,0
1,135089.0,90.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,-7.0,6.0,0.0,0.0,0,0,1,0
2,135089.0,90.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,-7.0,6.0,0.0,0.0,0,0,1,0
3,135089.0,90.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,-7.0,6.0,0.0,0.0,0,0,1,0
4,135089.0,90.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,-7.0,6.0,0.0,0.0,0,0,1,0


# Calc Sofa Score

In [19]:
test_df = base_test_df.copy()
column_list = test_df.columns.to_list()

# 1. **Respiration:** PaO2 / FiO2, Mechanical Ventilation(yes or no)
# 2. **Coagulation:** Platelets
# 3. **Liver:** Bilirubin
# 4. **Neurological:** Glasgow coma score(GCS)
# 5. **Cardiovascular:** MAP, Vasopressors(yes or no) -> MAP은 안보여서 MBP쓰자. SBP(침습성수축기동맥혈압), DBP(침습성확장기동맥혈압), MBP(침습성평균동맥혈압???)
# 6. **Renal:** Creatinine, Urine output
dummy = 'PaO2,FiO2,MechVent,Platelets,Bilirubin,GCS,MBP,Creatinine,Urine'
col_list = dummy.split(',')
col_list

print(len(test_df))
test_df2 = test_df[col_list].copy()
test_df2.isnull().sum()

print(len(test_df2))

test_df2.MechVent.value_counts()

28000
28000


1.0    25134
0.0     2866
Name: MechVent, dtype: int64

In [28]:
# 기준표에 따라 6개 항목에 대한 점수를 책정한다.
# 기준표를 실은 Notion Link: https://www.notion.so/99a8a4f45d9f423f97e4a41f5af2c974

cal_respi = test_df2.PaO2 / test_df2.FiO2
# Respiratory
respi = cal_respi.copy()
respi.name = 'PaO2/FiO2'
respi[:] = 0
respi[cal_respi < 400] = 1
respi[cal_respi < 300] = 2
respi[(cal_respi < 200) & (test_df2.MechVent == 1)] = 3
respi[(cal_respi < 100) & (test_df2.MechVent == 1)] = 4
# print(respi.value_counts())

coagulation = test_df2.Platelets.copy()
# print(coagulation.value_counts())
coagulation[:] = 0
coagulation[test_df2.Platelets < 150] = 1
coagulation[test_df2.Platelets < 100] = 2
coagulation[test_df2.Platelets < 50] = 3
coagulation[test_df2.Platelets < 20] = 4
coagulation.value_counts()

liver = test_df2.Bilirubin.copy()
liver[:] = 4
liver[(test_df2.Bilirubin >= 6.) & (test_df2.Bilirubin <= 11.9)] = 3
liver[(test_df2.Bilirubin >= 2.) & (test_df2.Bilirubin <= 5.9)] = 2
liver[(test_df2.Bilirubin >= 1.2) & (test_df2.Bilirubin <= 1.9)] = 1
liver[(test_df2.Bilirubin < 1.2)] = 0
liver[test_df2.Bilirubin == np.nan] = 0
liver.value_counts()

cardiovascular = test_df2.MBP.copy()
cardiovascular[:] = 0
cardiovascular[test_df2.MBP < 70] = 1

gcs = test_df2.GCS.copy()
gcs[:] = 0
# gcs[(test_df2.GCS == 15)] = 0
gcs[(test_df2.GCS >= 13.) & (test_df2.GCS <= 14.)] = 1
gcs[(test_df2.GCS >= 10.) & (test_df2.GCS <= 12.)] = 2
gcs[(test_df2.GCS >= 6.) & (test_df2.GCS <= 9.)] = 3
gcs[(test_df2.GCS < 6.)] = 4
gcs.value_counts()

renal = test_df2.Creatinine.copy()
# test_df2.Urine
renal[:] = 4
renal[(test_df2.Creatinine >= 3.5) & (test_df2.Creatinine <= 4.9)] = 3
renal[(test_df2.Creatinine >= 2.) & (test_df2.Creatinine <= 3.4)] = 2
renal[(test_df2.Creatinine >= 1.2) & (test_df2.Creatinine <= 1.9)] = 1
renal[(test_df2.Creatinine < 1.2)] = 0
renal[test_df2.Creatinine == np.nan] = 0
renal.value_counts()

sofa_score_df = pd.concat([respi, coagulation, liver, cardiovascular, gcs, renal], axis=1)


In [21]:
sofa_score_df.head()

,PaO2/FiO2,Platelets,Bilirubin,MBP,GCS,Creatinine
0,0.0,4.0,0.0,1.0,4.0,0.0
1,0.0,4.0,0.0,1.0,4.0,0.0
2,0.0,4.0,0.0,1.0,4.0,0.0
3,0.0,4.0,0.0,1.0,4.0,0.0
4,0.0,4.0,0.0,1.0,4.0,0.0


In [31]:
# 6가지 항목에 대해 각각 점수를 구하고, 이들을 합산하여 sofa_score를 구한다.
# 환자별로 처리하기 위해 pid 정보를,
# 환자별 실제 labeling 정보를 얻기 위해 labels 정보를 가져온다.

sofa_score_df['sofa_score'] = sofa_score_df.sum(axis=1)
sofa_score_df
sofa_score_df['pid'] = test_df.pid
sofa_score_df['labels'] = test_df.labels
sofa_score_df

,PaO2/FiO2,Platelets,Bilirubin,MBP,GCS,Creatinine,sofa_score,pid,labels
0,0.0,4.0,0.0,1.0,4.0,0.0,9.0,135089.0,0.0
1,0.0,4.0,0.0,1.0,4.0,0.0,9.0,135089.0,0.0
2,0.0,4.0,0.0,1.0,4.0,0.0,9.0,135089.0,0.0
3,0.0,4.0,0.0,1.0,4.0,0.0,9.0,135089.0,0.0
4,0.0,4.0,0.0,1.0,4.0,0.0,9.0,135089.0,0.0
...,...,...,...,...,...,...,...,...,...
27995,2.0,1.0,0.0,0.0,0.0,1.0,4.0,135219.0,0.0
27996,2.0,1.0,0.0,0.0,0.0,1.0,4.0,135219.0,0.0
27997,2.0,1.0,0.0,0.0,0.0,1.0,4.0,135219.0,0.0
27998,3.0,1.0,0.0,0.0,0.0,1.0,5.0,135219.0,0.0


In [32]:
sofa_score_df.to_csv('SOFA_df.csv', index=False)

In [23]:
# SOFA에서 환자별 sofa_score를 결정하기 위해 환자별 모든 데이터에 대해서 mean, max, median 중 무엇을 쓸거냐 결정
# 여기서는 모두 구해보고 가장 좋은 auc score를 내는 agg 함수를 채택한다.

sofa_df_max = sofa_score_df.groupby(by='pid').agg({'labels':np.median, 'sofa_score':np.max}).reset_index()
sofa_df_median = sofa_score_df.groupby(by='pid').agg({'labels':np.median, 'sofa_score':np.median}).reset_index()
sofa_df_mean = sofa_score_df.groupby(by='pid').agg({'labels':np.median, 'sofa_score':np.mean}).reset_index()

print(len(sofa_df_max))
print(len(sofa_df_mean))
print(len(sofa_df_median))
sofa_df_median.rename(columns={'sofa_score':'sofa_median_score'}, inplace=True)
sofa_df_median

400
400
400


,pid,labels,sofa_median_score
0,132551.0,1.0,6.0
1,132567.0,0.0,3.0
2,132570.0,0.0,6.0
3,132582.0,0.0,3.0
4,132584.0,0.0,8.0
5,132590.0,0.0,2.0
6,132595.0,0.0,9.0
7,132598.0,1.0,3.0
8,132610.0,0.0,4.0
9,132614.0,0.0,5.0


In [30]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(sofa_df_median.labels, sofa_df_median.sofa_median_score))

0.7049071618037135


In [29]:
from sklearn.metrics import roc_auc_score

# 모델의 train, valid, test data의 shape은 (batch_size, maxlen, feature_dim)
# 따라서 모델 출력의 shape은 (batch_size,) or (batch_size, 1)임.
# SOFA 모델에서도 환자별 사망 여부가 나와야 하기에 (batch_size, 1) 형식이 되어야 함.
# 따라서 'sofa_score_func'은 환자별로 모두 동일한 값을 넣었기 때문에 drop_duplicates()를 활용한다.
real_max = sofa_df_max.labels
pred_max = sofa_df_max.sofa_score
real_mean = sofa_df_mean.labels
pred_mean = sofa_df_mean.sofa_score
real_median = sofa_df_median.labels
pred_median = sofa_df_median.sofa_score

max_case = roc_auc_score(real_max, pred_max)
mean_case = roc_auc_score(real_mean, pred_mean)
median_case = roc_auc_score(real_median, pred_median)

print(f'sofa auc_roc_score(max_case): {max_case}')
print(f'sofa auc_roc_score(median_case): {median_case}')
print(f'sofa auc_roc_score(mean_case): {mean_case}')


AttributeError: ignored

In [26]:
print(len(sofa_df_median))
sofa_df_median

400


,pid,labels,sofa_median_score
0,132551.0,1.0,6.0
1,132567.0,0.0,3.0
2,132570.0,0.0,6.0
3,132582.0,0.0,3.0
4,132584.0,0.0,8.0
5,132590.0,0.0,2.0
6,132595.0,0.0,9.0
7,132598.0,1.0,3.0
8,132610.0,0.0,4.0
9,132614.0,0.0,5.0


In [ ]:
sofa_result = sofa_df_median.copy()
for i in range(len(sofa_df_median)):
    dup = [i]*69
    sofa_result = sofa_result.append(sofa_result.iloc[dup, :])
print(len(sofa_result))
sofa_result.sort_values(by='pid', ascending=True, inplace=True)
sofa_result.reset_index(drop=True, inplace=True)
sofa_result

28000


,pid,labels,sofa_median_score
0,132551.0,1.0,6.0
1,132551.0,1.0,6.0
2,132551.0,1.0,6.0
3,132551.0,1.0,6.0
4,132551.0,1.0,6.0
...,...,...,...
27995,142641.0,0.0,2.0
27996,142641.0,0.0,2.0
27997,142641.0,0.0,2.0
27998,142641.0,0.0,2.0


# Calc MEWS score
 - from MEWS_from_jaeeun.ipynb

In [ ]:
mews_df = pd.read_csv(path+'mews.csv')
mews_df.tail()

In [ ]:
sofa_df_median.tail()

# Calc NEWS score
- from news_from_yb.ipynb